In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
import matplotlib.pylab as plt

In [3]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [4]:
# Import des questions

In [5]:
answers_grand_debat_DEC_raw = pd.read_csv('raw/large_datasets_ELF/DEMOCRATIE_ET_CITOYENNETE.csv', encoding="utf-8").drop_duplicates(keep=False)
answers_grand_debat_FDP_raw = pd.read_csv('raw/large_datasets_ELF/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.csv', encoding="utf-8").drop_duplicates(keep=False)

/anaconda3/envs/elfrance/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
def keep_popular_hashtags_by_question(dataframe):
    for question in dataframe['id_elf'].unique():    
        allowed_categories_tuples = Counter(dataframe[dataframe['id_elf'] == question]['Categorie']).most_common(20)
        allowed_categories = [element[0] for element in allowed_categories_tuples]
        dataframe = dataframe[(dataframe['id_elf'] != 'question') | (dataframe['Categorie'].isin(allowed_categories))]
    return dataframe

def process_LGD_dataset(raw_dataframe):
    """
    """
    answers_annotees = pd.read_csv('raw/actions.csv', encoding="utf-8").rename(columns={'Contribution': 'reference'})
    raw_dataframe = pd.merge(answers_annotees, raw_dataframe, on='reference')
    print(raw_dataframe.shape)
    
    questions_dictionary = pd.read_csv('raw/match_questions_ELF_LGA.csv', header=0)
    raw_dataframe = pd.merge(raw_dataframe, questions_dictionary, left_on='Question', right_on='id_gda')
    print(raw_dataframe.shape)

    gda_dictionary = pd.read_csv('raw/matching_gda_id1_id2.csv', header=0)
    raw_dataframe = pd.merge(raw_dataframe, gda_dictionary, left_on='id_gda', right_on='id1_gda')
    print(raw_dataframe.shape)
    
    raw_dataframe["corpus"] = None
    
    for index, row in raw_dataframe.iterrows():
        raw_dataframe.at[index,'corpus'] = raw_dataframe.loc[index][raw_dataframe.filter(like=row['id2_gda']).columns[0]]

    raw_dataframe = raw_dataframe[['reference', 'id_elf', 'corpus', 'Categorie', 'Annotateur']]
    
    removed_categories = ['Z Autres', 'Sans réponse / Hors sujet / Inclassable']
    raw_dataframe = raw_dataframe[~raw_dataframe['Categorie'].isin(removed_categories)]
    print(raw_dataframe.shape)
    
    raw_dataframe = keep_popular_hashtags_by_question(raw_dataframe)
    print(raw_dataframe.shape)
    
    raw_dataframe = pd.DataFrame({'count' : raw_dataframe.groupby( ['reference', 'Categorie', 'corpus', 'id_elf'] ).apply(lambda x: x.Annotateur.nunique())}).reset_index()
    print(raw_dataframe.shape)
    
    return raw_dataframe.rename(columns={'reference': 'contribution', 'Categorie': 'tag', 'id_elf' : 'question'})
    

In [33]:
answers_grand_debat_GDA_FDP = process_LGD_dataset(answers_grand_debat_FDP_raw)
answers_grand_debat_GDA_DEC = process_LGD_dataset(answers_grand_debat_DEC_raw)

(302698, 24)
(23033, 26)
(23033, 28)
(18943, 5)
(18943, 5)
(8857, 5)
(56677, 53)
(5918, 55)
(5918, 57)
(4679, 5)
(4679, 5)
(1883, 5)


In [34]:
answers_grand_debat_GDA = pd.concat([answers_grand_debat_GDA_FDP, answers_grand_debat_GDA_DEC])
answers_grand_debat_GDA['source'] = 'GDA'

In [35]:
answers_grand_debat_ELF = pd.read_csv("preprocessed/elf_tags_sample.csv", sep=";")
answers_grand_debat_ELF['source'] = 'ELF'

In [36]:
answers_grand_debat = pd.concat([answers_grand_debat_ELF, answers_grand_debat_GDA])
answers_grand_debat_input.to_csv('preprocessed/reponses_annotees_multilabel.csv', sep='\t', encoding='utf-8')

In [37]:
answers_grand_debat.shape

(11315, 6)

In [38]:
answers_grand_debat.head()

,contribution,tag,corpus,question,count,source
0,37 240,Le peuple pas responsable du problème écologique,Stopper toutes ces taxes et donner une autre i...,TE13,1,ELF
1,12 970,Etre plus à l'écoute,Avoir des représentants actif qui remontent et...,OE11,1,ELF
2,12 970,Plus de transparence,Avoir des représentants actif qui remontent et...,OE11,1,ELF
3,13 210,Incitations financières au lieu de taxes,Il faudrait pousser la création de beaucoup pl...,TE13,1,ELF
4,13 210,Développer les énergies renouvelables,Il faudrait pousser la création de beaucoup pl...,TE13,1,ELF
